# 脚本编程与系统管理

首先补充一些基本的在python脚本中执行系统命令的代码。cank 

In [53]:
import os
os.system('ls')

1

In [54]:
tmp = os.popen('ls *.py').readlines()
tmp

[]

In [55]:
import subprocess

p = subprocess.Popen('ls', shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print (line)
retval = p.wait()

b"'ls' \xb2\xbb\xca\xc7\xc4\xda\xb2\xbf\xbb\xf2\xcd\xe2\xb2\xbf\xc3\xfc\xc1\xee\xa3\xac\xd2\xb2\xb2\xbb\xca\xc7\xbf\xc9\xd4\xcb\xd0\xd0\xb5\xc4\xb3\xcc\xd0\xf2\r\n"
b'\xbb\xf2\xc5\xfa\xb4\xa6\xc0\xed\xce\xc4\xbc\xfe\xa1\xa3\r\n'


## 命令行解析器

Python 命令行与参数解析方法有很多工具，这里学习使用python 自带的argparse ，来说明python 如何进行命令行解析。主要参考了

- [HelloGitHub-Article](https://github.com/HelloGitHub-Team/Article)
- [argparse模块用法实例详解](https://zhuanlan.zhihu.com/p/56922793)
- python[官方文档](https://docs.python.org/zh-cn/3/library/argparse.html)
- [Python-argparse-命令行与参数解析](https://zhuanlan.zhihu.com/p/34395749)

通俗来说，命令行与参数解析就是当你输入cmd 打开dos 交互界面时候，启动程序要进行的参数给定。比如在dos 界面输入：

```code
python openPythonFile.py "a" -b "number"
```

其中，"a" -b等就是命令行与参数解析要做的事情。先不用深究参数的含义，这里就是个示例，简而言之，就是设计程序在**运行时必须给定某些额外参数**才能运行，也就是如果设置了命令行参数解析，那么各种编译器按F5 是无法直接运行程序的。这样的目的之一是不能随便就能运行脚本，可以达到一定程度上的安全功能。

那肯定就会好奇命令行中敲入一段命令后，是如何被解析执行的？自己如何实现一个命令行工具来帮助执行和处理任务？如何利用python库来帮助实现？

这一节就主要记录如何使用Python 内置的 argparse 标准库解析命令行。

argparse 作为 Python 内置的标准库，提供了较为简单的方式来编写命令行接口。当你在程序中定义需要哪些参数，argparse 便会从 sys.argv 中获取命令行输入进行解析，对正确或非法输入做出响应，也可以自动生成帮助信息和使用说明。

总体上分为三大步：

- 创建解析：设置解析器，后续对命令行的解析就**依赖于这个解析器**，它能够**将命令行字符串转换为Python对象**。通过实例化 argparse.ArgumentParser，给定一些选填参数，就可以设置一个解析器
- 添加参数：通过ArgumentParser.add_argument 方法来**为解析器设置参数信息**，以告诉解析器命令行字符串中的**哪些内容**应解析为**哪些类型的Python对象**。注意，每一个参数都要单独设置，需要两个参数就用两个add_argument
- 解析参数：定义好参数后，就可以使用 ArgumenteParser.**parse_args 方法来解析一组命令行参数字符串**了。默认情况下，参数取自**sys.argv[1:]**,它就是我们在命令行敲入的**一段命令（不含文件名）所对应的一个字符串列表**，比如，若输入 python3 cmd.py --sum 1 2 3，那么sys.argsv[1:]就是['--sum','1','2','3']。

基本的业务逻辑是这样的。解析好命令行后，我们就可以从解析结果中获取每个参数的值，进而根据自己的业务需求做进一步的处理。比如，对于上文中所定义的nums参数，我们可以通过解析后的结果中的accumulate方法对其进行求最大值或求和（取决于是否提供 --sum 参数）。

下面就给出一个较完整的代码示例。

In [56]:
import argparse

# 1. 设置解析器
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)

# 2. 定义参数
# 添加 nums 参数，在使用信息中显示为 num
# 其类型为 int，且支持输入多个，且至少需要提供一个
parser.add_argument('nums',  metavar='num', type=int, nargs='+',
                    help='a num for the accumulator')
# 添加 --sum 参数，该参数被 parser 解析后所对应的属性名为 accumulate
# 若不提供 --sum，默认值为 max 函数，否则为 sum 函数
parser.add_argument('--sum', dest='accumulate', action='store_const',
                    const=sum, default=max,
                    help='sum the nums (default: find the max)')

# 3. 解析命令行
args = parser.parse_args(['--sum', '-1', '0', '1'])
print(args) # 结果：Namespace(accumulate=<built-in function sum>, nums=[-1, 0, 1])

# 4. 业务逻辑
result = args.accumulate(args.nums)
print(result)

Namespace(nums=[-1, 0, 1], accumulate=<built-in function sum>)
0


若我们需要对一组数字求和，只需执行：

```Shell
$ python3 cmd.py --sum -1 0 1
0
```

若需要对一组数字求最大值，只需执行：

```Shell
$ python3 cmd.py -1 0 1
1
```

如果给定的参数不是数字，则会报错提示：

```Shell
$ python3 cmd.py a b c
usage: cmd.py [-h] [--sum] num [num ...]
cmd.py: error: argument num: invalid int value: 'a'
```

我们还可以通过 -h 或 --help 参数查看其自动生成的使用说明和帮助：

```Shell
usage: cmd.py [-h] [--sum] num [num ...]

My Cmd Line Program

positional arguments:
  num         a num for the accumulator

optional arguments:
  -h, --help  show this help message and exit
  --sum       sum the nums (default: find the max)
```

接下来进一步探讨关于argparse更多复杂的情况，比如各种类型参数、参数前缀、参数组、互斥选项、嵌套解析、自定义帮助等等。主要要认识的问题是：argparse支持哪些类型的参数？这些参数该如何配置？

In [57]:
import argparse

# 1. 设置解析器
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)

1. 参数动作

In [58]:
parser.add_argument('--sum', dest='accumulate', action='store_const',
                    const=sum, default=max,
                    help='sum the nums (default: find the max)')

_StoreConstAction(option_strings=['--sum'], dest='accumulate', nargs=0, const=<built-in function sum>, default=<built-in function max>, type=None, choices=None, help='sum the nums (default: find the max)', metavar=None)

这里面的 action，也就是 参数动作，究竟是用来做什么的呢？

想象一下，当我们在命令行输入**一串参数**后，对于**不同类型的参数是希望做不同的处理**的。 那么 **参数动作** 其实就是告诉解析器，我们希望**对应的参数该被如何处理**。比如，参数值是该被存成一个值呢，还是追加到一个列表中？是当成布尔的 True 呢，还是 False？

参数动作 被分成了如下 8 个类别：

- store —— 保存参数的值，这是**默认**的参数动作。它通常用于给一个参数指定值，如指定名字：

In [59]:
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)
parser.add_argument('--name')
parser.parse_args(['--name', 'Eric'])

Namespace(name='Eric')

- store_const —— 保存被 const 命名的固定值。当我们想通过**是否给定参数**来起到**标志**的作用，给定就取某个值，就可以使用该参数动作，如：

In [60]:
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)
parser.add_argument('--sum', action='store_const', const=sum)
parser.parse_args(['--sum'])

Namespace(sum=<built-in function sum>)

- store_true 和 store_false —— 是 store_const 的特殊情况，用来分别保存 True 和 False。如果为指定参数，则其默认值分别为 False 和 True，如：

In [61]:
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)
parser.add_argument('--use', action='store_true')
parser.add_argument('--nouse', action='store_false')
parser.parse_args(['--use', '--nouse'])

Namespace(use=True, nouse=False)

In [62]:
parser.parse_args([])

Namespace(use=False, nouse=True)

- append —— 将参数值追加保存到一个列表中。它常常用于命令行中允许多个相同选项，如：

In [63]:
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)
parser.add_argument('--file', action='append')
parser.parse_args(['--file', 'f1', '--file', 'f2'])

Namespace(file=['f1', 'f2'])

- append_const —— 将 const 命名的固定值追加保存到一个列表中（const 的默认值为 None）。它常常用于将多个参数所对应的固定值都保存在同一个列表中，相应的需要 dest 入参来配合，以放在同一个列表中，如：

不指定 dest 入参，则固定值保存在以参数名命名的变量中

In [64]:
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)
parser.add_argument('--int', action='append_const', const=int)
parser.add_argument('--str', action='append_const', const=str)
parser.parse_args(['--int', '--str'])

Namespace(int=[<class 'int'>], str=[<class 'str'>])

指定 dest 入参，则固定值保存在 dest 命名的变量中

In [65]:
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)
parser.add_argument('--int', dest='types', action='append_const', const=int)
parser.add_argument('--str', dest='types', action='append_const', const=str)
parser.parse_args(['--int', '--str'])

Namespace(types=[<class 'int'>, <class 'str'>])

- count —— 计算参数出现次数，如：

In [66]:
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)
parser.add_argument('--increase', '-i', action='count')
parser.parse_args(['--increase', '--increase'])

Namespace(increase=2)

In [67]:
parser.parse_args(['-iii'])

Namespace(increase=3)

- help —— 打印解析器中所有选项和参数的完整帮助信息，然后退出。

- version —— 打印命令行版本，通过指定 version 入参来指定版本，调用后退出。如：

In [68]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--version', action='version', version='%(prog)s 1.0')
parser.parse_args(['--version'])

CMD 1.0


SystemExit: 0

C:\Users\hust2\miniconda3\envs\hydrus\lib\site-packages\IPython\core\interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


2. 参数类别

如果说 参数动作 定义了**解析器在接收到参数后该如何处理参数**，那么 参数类别 就是告诉解析器**这个参数的元信息**，也就是参数是什么样的。比如，参数是字符串呢？还是布尔类型呢？参数是在几个值中可选的呢？还是可以给定值，等等。

可选参数 顾名思义就是参数是可以加上，或不加上。**默认**情况下，通过 ArgumentParser.add_argument 添加的参数就是**可选参数**。

可以通过 - 来指定**短参数**，也就是名称短的参数；也可以通过 -- 来指定**长参数**，也就是名称长的参数。当然也可以两个都指定。

可选参数通常用于：用户提供一个参数以及对应值，则使用该值；若不提供，则使用默认值。如：

In [69]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--name', '-n')
parser.parse_args(['--name', 'Eric'])  # 通过长参数指定名称

Namespace(name='Eric')

In [70]:
parser.parse_args(['-n', 'Eric']) # 通过短参数指定名称

Namespace(name='Eric')

In [71]:
parser.parse_args([]) # 不指定则默认为 None

Namespace(name=None)

参数类型 就是解析器**参数值是要作为什么类型去解析**，默认情况下是 str 类型。我们可以通过 type 入参来指定参数类型。

argparse 所支持的参数类型多种多样，可以是 int、float、bool等，比如：

In [72]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('-i', type=int)
parser.add_argument('-f', type=float)
parser.add_argument('-b', type=bool)
parser.parse_args(['-i', '1', '-f', '2.1', '-b', '0'])

Namespace(i=1, f=2.1, b=True)

更厉害的是，type 入参还可以是**可调用(callable)对象**。这就给了我们很大的想象空间，可以指定 type=open 来把参数值作为文件进行处理，也可以指定自定义函数来进行类型检查和类型转换。

作为文件进行处理：

In [73]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--file', type=open)
parser.parse_args(['--file', 'test.txt'])

Namespace(file=<_io.TextIOWrapper name='test.txt' mode='r' encoding='cp936'>)

使用自定义函数进行处理，入参为参数值，需返回转换后的结果。 比如，对于参数 --num，我们希望当其值小于 1 时则返回 1，大于 10 时则返回 10：

In [74]:
def limit(string):
    num = int(string)
    if num < 1:
        return 1
    if num > 10:
        return 10
    return num
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--num', type=limit)
parser.parse_args(['--num', '-1'])  # num 小于1，则取1

Namespace(num=1)

In [75]:
parser.parse_args(['--num', '15'])  # num 大于10，则取10

Namespace(num=10)

In [76]:
parser.parse_args(['--num', '5'])  # num 在1和10之间，则取原来的值

Namespace(num=5)

3. 参数默认值

参数默认值 用于在命令行中不传参数值的情况下的默认取值，可通过 default 来指定。如果不指定该值，则参数默认值为 None。

比如：

In [77]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('-i', default=0, type=int)
parser.add_argument('-f', default=3.14, type=float)
parser.add_argument('-b', default=True, type=bool)
parser.parse_args([])

Namespace(i=0, f=3.14, b=True)

4. 位置参数

位置参数 就是通过位置而非是 - 或 -- 开头的参数来指定参数值。

比如，我们可以指定两个位置参数 x 和 y ，先添加的 x 位于第一个位置，后加入的 y 位于第二个位置。那么在命令行中输入 1 2的时候，分别对应到的就是 x 和 y：

In [78]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('x')
parser.add_argument('y')
parser.parse_args(['1', '2'])

Namespace(x='1', y='2')

5. 可选值

可选值 就是**限定参数值的内容**，通过 choices 入参指定。

有些情况下，我们可能需要限制用户输入参数的内容，只能在预设的几个值中选一个，那么 可选值 就派上了用场。

比如，指定文件读取方式限制为 read-only 和 read-write：

In [79]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--mode', choices=('read-only', 'read-write'))
parser.parse_args(['--mode', 'read-only'])

Namespace(mode='read-only')

In [80]:
parser.parse_args(['--mode', 'read'])

usage: CMD [-h] [--mode {read-only,read-write}]
CMD: error: argument --mode: invalid choice: 'read' (choose from 'read-only', 'read-write')
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 1851, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 2060, in _parse_known_args
    start_index = consume_optional(start_index)
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 2000, in consume_optional
    take_action(action, args, option_string)
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 1912, in take_action
    argument_values = self._get_values(action, argument_strings)
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 2444, in _get_values
    self._check_value(action, value)
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 2500, in _check_value
    raise ArgumentError(action, msg % args)
argparse.ArgumentError: argument --mode: invalid choice: 'read' (choose from 'read-only', 'read-write')

Du

TypeError: object of type 'NoneType' has no len()

6. 互斥参数

互斥参数 就是多个参数之间彼此互斥，不能同时出现。使用互斥参数首先通过 ArgumentParser.add_mutually_exclusive_group 在解析器中添加一个互斥组，然后在这个组里添加参数，那么组内的所有参数都是互斥的。

比如，我们希望通过命令行来告知乘坐的交通工具，要么是汽车，要么是公交，要么是自行车，那么就可以这么写：

In [81]:
parser = argparse.ArgumentParser(prog='CMD')
group = parser.add_mutually_exclusive_group()
group.add_argument('--car', action='store_true')
group.add_argument('--bus', action='store_true')
group.add_argument('--bike', action='store_true')
parser.parse_args([])  # 什么都不乘坐

Namespace(car=False, bus=False, bike=False)

In [82]:
parser.parse_args(['--bus'])  # 乘坐公交

Namespace(car=False, bus=True, bike=False)

In [83]:
parser.parse_args(['--bike'])  # 骑自行车

Namespace(car=False, bus=False, bike=True)

In [84]:
parser.parse_args(['--bike', '--car'])  # 又想骑车，又想坐车，那是不行的

usage: CMD [-h] [--car | --bus | --bike]
CMD: error: argument --car: not allowed with argument --bike
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 1851, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 2060, in _parse_known_args
    start_index = consume_optional(start_index)
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 2000, in consume_optional
    take_action(action, args, option_string)
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 1923, in take_action
    raise ArgumentError(action, msg % action_name)
argparse.ArgumentError: argument --car: not allowed with argument --bike

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\hust2\AppDat

TypeError: object of type 'NoneType' has no len()

7. 可变参数列表

可变参数列表 用来定义一个参数可以有多个值，且能通过 nargs 来定义值的个数。

若 nargs=N，N为一个数字，则要求该参数提供 N 个值，如：

In [85]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--foo', nargs=2)
print(parser.parse_args(['--foo', 'a', 'b']))

Namespace(foo=['a', 'b'])


若 nargs=?，则要求改参数提供 0 或 1 个值，如：

In [86]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--foo', nargs='?')
parser.parse_args(['--foo'])

Namespace(foo=None)

In [87]:
parser.parse_args(['--foo', 'a'])

Namespace(foo='a')

若 nargs=*，则要求改参数提供 0 或多个值，如：

In [88]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--foo', nargs='*')
parser.parse_args(['--foo', 'a', 'b', 'c', 'd', 'e'])

Namespace(foo=['a', 'b', 'c', 'd', 'e'])

若 nargs=+，则要求改参数至少提供 1 个值，如：

In [89]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--foo', nargs='+')
parser.parse_args(['--foo', 'a'])

Namespace(foo=['a'])

小结下。

add_argument 方法定义**单个**的命令行参数应当**如何解析**。每个形参更多的描述：

- name or flags - 一个命名或者一个选项字符串的列表，例如 foo 或 -f, --foo。
- action - 当参数在命令行中出现时使用的动作基本类型。
- nargs - 命令行参数应当消耗的数目。
- const - 被一些 action 和 nargs 选择所需求的常数。
- default - 当参数未在命令行中出现时使用的值。
- type - 命令行参数应当被转换成的类型。
- choices - 可用的参数的容器。
- required - 此命令行选项是否可省略 （仅选项可用）。
- help - 一个此选项作用的简单描述。
- metavar - 在使用方法消息中使用的参数值示例。
- dest - 解析后的参数名称，默认情况下，对于可选参数选取最长的名称，中划线转换为下划线. 

然后一个比较完整的，需要在命令行中执行的例子如下，对应的python文件是argv_argparse.py.

调用方式：

```Shell
python argv_argparse.py -h
python argv_argparse.py xiaoming 1991.11.11
python argv_argparse.py xiaoming 1991.11.11 -p xiaohong xiaohei -a 25 -r han -s female -o 1 2 3 4 5 6
```

-h表示调出help信息。

以上是参数动作和参数类别相关内容，接下来继续深入了解 argparse 的功能，包括如何修改参数前缀，如何定义参数组，如何定义嵌套的解析器，如何编写自定义动作等。

1. 帮助

自动生成帮助

当你在命令行程序中指定 -h 或 --help 参数时，都会输出帮助信息。而 argparse 可通过指定 add_help 入参为 True 或不指定，以达到自动输出帮助信息的目的。

In [90]:
import argparse
parser = argparse.ArgumentParser(add_help=True)
parser.add_argument('--foo')
parser.parse_args(['-h'])

usage: ipykernel_launcher.py [-h] [--foo FOO]

optional arguments:
  -h, --help  show this help message and exit
  --foo FOO


SystemExit: 0

自定义帮助

ArgumentParser 使用 formatter_class 入参来控制所输出的帮助格式。 比如，通过指定 formatter_class=argparse.RawTextHelpFormatter，我们可以让帮助内容遵循原始格式：

In [91]:
import argparse
parser = argparse.ArgumentParser(
    add_help=True,
    formatter_class=argparse.RawTextHelpFormatter,
    description="""description  raw formatted"""
)
parser.add_argument(
    '-a', action="store_true",
    help="""argument raw formatted"""
)
parser.parse_args(['-h'])

usage: ipykernel_launcher.py [-h] [-a]

description  raw formatted

optional arguments:
  -h, --help  show this help message and exit
  -a          argument raw formatted


SystemExit: 0

2. 参数组

有时候，我们需要给参数分组，以使得在显示帮助信息时能够显示到一起。

比如某命令行支持三个参数选项 --user、--password和--push，前两者需要放在一个名为 authentication 的分组中以表示它们是身份认证信息。那么我们可以用 ArgumentParser.add_argument_group 来满足：

In [92]:
import argparse
parser = argparse.ArgumentParser()
group = parser.add_argument_group('authentication')
group.add_argument('--user', action="store")
group.add_argument('--password', action="store")
parser.add_argument('--push', action='store')
parser.parse_args(['-h'])

usage: ipykernel_launcher.py [-h] [--user USER] [--password PASSWORD] [--push PUSH]

optional arguments:
  -h, --help           show this help message and exit
  --push PUSH

authentication:
  --user USER
  --password PASSWORD


SystemExit: 0

3. 选项参数前缀

不知你是否注意到，在不同平台上命令行程序的选项参数前缀可能是不同的。比如在 Unix 上，其前缀是 -；而在 Windows 上，大多数命令行程序（比如 findstr）的选项参数前缀是 /。

在 argparse 中，选项参数前缀默认采用 Unix 命令行约定，也就是 -。但它也支持自定义前缀，下面是一个例子：

In [93]:
import argparse
parser = argparse.ArgumentParser(
    description='Option prefix',
    prefix_chars='-+/',
)

parser.add_argument('-power', action="store_false",
                    default=None,
                    help='Set power off',
                   )
parser.add_argument('+power', action="store_true",
                    default=None,
                    help='Set power on',
                   )
parser.add_argument('/win',
                    action="store_true",
                    default=False)
parser.parse_args(['-power'])

Namespace(power=False, win=False)

In [94]:
parser.parse_args(['+power', '/win'])

Namespace(power=True, win=True)

在这个例子中，我们指定了三个选项参数前缀 -、+和/，从而：

- 通过指定选项参数 -power，使得 power=False
- 通过指定选项参数 +power，使得 power=True
- 通过指定选项参数 /win，使得 win=True

## 读取配置文件

很多情况下，我们需要通过配置文件来定义一些参数性质的数据，因为配置文件作为一种可读性很好的格式，非常适用于存储程序中的配置数据。 

在每个配置文件中，配置数据会被分组（比如例子中的“installation”、 “debug” 和 “server”）。 每个分组在其中指定对应的各个变量值。那么如何读取普通.ini格式的配置文件？

在python中，configparser 模块能被用来读取配置文件。例如，假设有配置文件config.ini。下面给出读取代码：

In [95]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read('config.ini')

['config.ini']

In [96]:
cfg.sections()

['installation', 'debug', 'server']

In [97]:
cfg.get('installation','library')

'/usr/local/lib'

In [98]:
cfg.getboolean('debug','log_errors')

True

In [99]:
cfg.getint('server','port')

8080

In [100]:
cfg.getint('server','nworkers')

32

In [101]:
print(cfg.get('server','signature'))


Brought to you by the Python Cookbook


还可以读取一个section下的所有keys或所有键值对，参考：[Python 读取写入配置文件 —— ConfigParser](https://blog.csdn.net/jiede1/article/details/79064780)

In [102]:
cfg.options("installation") 

['library', 'include', 'bin', 'prefix']

In [103]:
cfg.items("installation")  

[('library', '/usr/local/lib'),
 ('include', '/usr/local/include'),
 ('bin', '/usr/local/bin'),
 ('prefix', '/usr/local')]

如果需要，还能修改配置并使用 cfg.write() 方法将其写回到文件中。例如：

In [104]:
cfg.set('server','port','9000')
cfg.set('debug','log_errors','False')
import sys
cfg.write(sys.stdout)

[installation]
library = %(prefix)s/lib
include = %(prefix)s/include
bin = %(prefix)s/bin
prefix = /usr/local

[debug]
log_errors = False
show_warnings = False

[server]
port = 9000
nworkers = 32
pid-file = /tmp/spam.pid
root = /www/root
signature = 
	Brought to you by the Python Cookbook

